<a href="https://colab.research.google.com/github/hopesofbuzzy/URFU_adii/blob/main/%D0%9E%D0%9F%D0%94/LaBSE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# Установка (если ещё не сделано)
!pip install -q sentence-transformers scikit-learn numpy

from sentence_transformers import SentenceTransformer
from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

texts = [
    "На улице Ленина образовалась большая яма, мешает проезду.",
    "Яма на проспекте Ленина стала причиной ДТП.",
    "В парке Горького сломана детская площадка.",
    "Детская площадка в парке Горького требует ремонта.",
    "Проблемы с водоснабжением в районе Ново-Переделкино.",
    "В Ново-Переделкино снова отключили горячую воду.",
    "Опять отрубили горячую воду в НВК, гады! Безбразие в УрФУ",
    "В Городской клинической больнице №40 открылся новый современный центр МРТ-диагностики. Запись уже доступна.",
    "Больница №40 получила новейший аппарат МРТ. Это сократит очереди на обследование.",
    "Дискокот вновь уведомил Славу о сикс-севен. 'Это просто имба', - считает Слава"
]

print("Исходные тексты:")
for i, t in enumerate(texts):
    print(f"{i+1}. {t}")

# Загрузка LaBSE
print("\nЗагружаем LaBSE...")
model = SentenceTransformer('sentence-transformers/LaBSE')

# Эмбеддинги (нормализованные — важно для cosine)
embeddings = model.encode(texts, normalize_embeddings=True)

# Сходство → принудительно ограничиваем [0, 1]
similarity_matrix = cosine_similarity(embeddings)
similarity_matrix = np.clip(similarity_matrix, 0.0, 1.0)  # 🔑 КЛЮЧЕВОЕ ИСПРАВЛЕНИЕ

# Расстояние
distance_matrix = 1.0 - similarity_matrix

# Кластеризация
clustering = DBSCAN(
    metric='precomputed',
    eps=0.35,
    min_samples=2
).fit(distance_matrix)

labels = clustering.labels_

# Вывод
print("\nРезультаты кластеризации:")
for i, (text, label) in enumerate(zip(texts, labels)):
    cluster = f"Кластер {label}" if label != -1 else "Шум"
    print(f"{i+1}. {cluster} → {text}")

Исходные тексты:
1. На улице Ленина образовалась большая яма, мешает проезду.
2. Яма на проспекте Ленина стала причиной ДТП.
3. В парке Горького сломана детская площадка.
4. Детская площадка в парке Горького требует ремонта.
5. Проблемы с водоснабжением в районе Ново-Переделкино.
6. В Ново-Переделкино снова отключили горячую воду.
7. Опять отрубили горячую воду в НВК, гады! Безбразие в УрФУ
8. В Городской клинической больнице №40 открылся новый современный центр МРТ-диагностики. Запись уже доступна.
9. Больница №40 получила новейший аппарат МРТ. Это сократит очереди на обследование.
10. Дискокот вновь уведомил Славу о сикс-севен. 'Это просто имба', - считает Слава

Загружаем LaBSE...

Результаты кластеризации:
1. Кластер 0 → На улице Ленина образовалась большая яма, мешает проезду.
2. Кластер 0 → Яма на проспекте Ленина стала причиной ДТП.
3. Кластер 1 → В парке Горького сломана детская площадка.
4. Кластер 1 → Детская площадка в парке Горького требует ремонта.
5. Кластер 2 → Проблемы 